# Classification 10/20/22

Goal: Start classifying tidydata_1020222 using sktime module



In [19]:
import json
import datetime
import pandas as pd
import numpy as np
import sys
import os
import plotly.express as px
from sktime.classification.kernel_based import RocketClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
df = pd.read_csv("../constants/tidydata_102022.csv", )
df

,DateTime,Temp C,RH %,Room,Window Open
0,2022-07-20 07:20:00,23.189,51.268,0,1
1,2022-07-20 07:20:30,23.189,51.362,0,1
2,2022-07-20 07:21:00,23.189,51.488,0,1
3,2022-07-20 07:21:30,23.189,51.457,0,1
4,2022-07-20 07:22:00,23.165,51.517,0,1
...,...,...,...,...,...
123601,2022-09-11 07:58:00,29.196,50.861,1,0
123602,2022-09-11 07:58:30,29.196,50.861,1,0
123603,2022-09-11 07:59:00,29.196,50.796,1,0
123604,2022-09-11 07:59:30,29.196,50.796,1,0


In [32]:
x = df.iloc[:, 1:4]
x

,Temp C,RH %,Room
0,23.189,51.268,0
1,23.189,51.362,0
2,23.189,51.488,0
3,23.189,51.457,0
4,23.165,51.517,0
...,...,...,...
123601,29.196,50.861,1
123602,29.196,50.861,1
123603,29.196,50.796,1
123604,29.196,50.796,1


In [33]:
x = x.to_numpy()
x

array([[23.189, 51.268,  0.   ],
       [23.189, 51.362,  0.   ],
       [23.189, 51.488,  0.   ],
       ...,
       [29.196, 50.796,  1.   ],
       [29.196, 50.796,  1.   ],
       [29.196, 50.796,  1.   ]])

In [34]:
y = df["Window Open"]
y = y.to_numpy()
y

array([1, 1, 1, ..., 0, 0, 0])

In [35]:

X_train, X_test, y_train, y_test = train_test_split(x, y,test_size=0.2, random_state=42)

In [36]:
X_train

array([[24.731, 52.678,  1.   ],
       [29.897, 48.431,  1.   ],
       [24.103, 57.157,  1.   ],
       ...,
       [24.997, 49.381,  0.   ],
       [25.288, 51.742,  1.   ],
       [29.872, 49.011,  1.   ]])

In [37]:
y_test

array([0, 0, 0, ..., 1, 0, 0])

In [38]:
# data tooo large -> 82 mins running and still not done 
# rocket = RocketClassifier()
# rocket.fit(X_train, y_train)
# y_pred = rocket.predict(X_test)
# accuracy_score(y_test, y_pred)

# try to sample the data instead....